In [14]:
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

In [204]:
url = 'https://preppykitchen.com/wprm_print/cream-pie'
url = 'https://preppykitchen.com/wprm_print/buttermilk-pie'
try:
    # Send a GET request to the base page
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Error fetching {url}: {e}")
    
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')


In [92]:
# Extract the title
title = soup.find('h2')  # The title is typically in an <h1> tag
if title:
    title_text = title.get_text().strip()
else:
    title_text = "Title not found"

# Extract the image URL
image = soup.find('img')  # The image is typically in an <img> tag
if image and 'src' in image.attrs:
    image_url = image['src']
else:
    image_url = "Image not found"

# Print the extracted information
print("Title:", title_text)
print("Image URL:", image_url)

Title: Cream Pie
Image URL: https://preppykitchen.com/wp-content/uploads/2019/04/Cream-Pie-recipe-330x330.jpg


In [93]:
# Extract the rating div
rating_div = soup.find('div', id='wprm-recipe-user-rating-0')

# Check if the div exists and extract the 'data-average' attribute
if rating_div and 'data-average' in rating_div.attrs:
    rating = rating_div['data-average']
    nbOfRatings = rating_div['data-count']  
    print(f"Rating: {rating} of {nbOfRatings}")  # Output: Rating: 5
else:
    print("Rating not found")
    
soup.find('div', id='wprm-recipe-user-rating-0')['data-average']


Rating: 5 of 42


'5'

In [208]:
def extract_time(time_string):
    """Convert time string to HH:MM:SS format"""
    if not time_string:
        return None
    minutes = int(re.search(r'\d+', time_string).group())
    return f"{minutes//60:02d}:{minutes%60:02d}:00"

from fractions import Fraction

def convert_fraction_text(quantity_text):
    # Define a mapping from fraction symbols to numeric string equivalents
    fraction_map = {
        '¼': '.25',
        '½': '.5',
        '¾': '.75',
        '⅓': '.33',
        '⅔': '.66',
        '⅕': '.2',
        '⅖': '.4',
        '⅗': '.6',
        '⅘': '.8'
    }

    # Check if the quantity text is in the mapping
    firstNumber = '0'
    if quantity_text[0] not in fraction_map:
        firstNumber = quantity_text[0]
    for number in quantity_text:
        if number in fraction_map:
            quantity_text =  firstNumber + fraction_map[number]
    
    
    #Treat fractions 
    quantity_list = quantity_text.split()
    #We need to treat examples like '1 1/2'
    if(len(quantity_list) > 1):
        sum = 0
        for quantity in quantity_list:
            sum += Fraction(quantity)
        quantity_text = str(sum)  
         
    
        
    return Fraction(quantity_text)

def extract_ingredients(soup):
    """Extract ingredients information with fractional quantities"""
    ingredients_list = []
    ingredients_container = soup.find('div', class_='wprm-recipe-ingredients-container')

    if ingredients_container:
        # ingredient_items = ingredients_container.find_all('li', class_='wprm-recipe-ingredient')
        groups_ingredients = ingredients_container.find_all('div', class_='wprm-recipe-ingredient-group')

        for group in groups_ingredients:
            groupname = group.find('h4', class_='wprm-recipe-group-name')
            ingredient_items = group.find_all('li', class_='wprm-recipe-ingredient')
            for item in ingredient_items:
                amount = item.find('span', class_='wprm-recipe-ingredient-amount')
                unit = item.find('span', class_='wprm-recipe-ingredient-unit')
                name = item.find('span', class_='wprm-recipe-ingredient-name')
                notes = item.find('span', class_='wprm-recipe-ingredient-notes')

                # Handle fractional quantities (like ½, ¼)
                if amount:
                    quantity_text = amount.text.strip()
                    try:
                        # Convert the fraction to a float if it's a valid fraction
                        quantity_text = convert_fraction_text(quantity_text)
                        quantity = float(Fraction(quantity_text))

                    except ValueError:
                        # If it's not a valid fraction (or not a number), keep it as None
                        print(f"it's not a valid fraction (or not a number), keep it as None")
                        quantity = None
                else:
                    quantity = None

                ingredient_dict = {
                    "ingredient": {
                        "name": name.text.strip() if name else None,
                        "nutrition": None
                    },
                    "quantity": quantity,
                    "unit": unit.text.strip() if unit else "whole",
                    "groupName" : groupname.text if groupname else None,
                    "notes": notes.text.strip() if notes else None
                }
                ingredients_list.append(ingredient_dict)

    return ingredients_list

def extract_instructions(soup):
    """Extract cooking instructions"""
    instructions_container = soup.find('div', class_='wprm-recipe-instructions-container')
    instructions = []
    
    groups_instructions = instructions_container.find_all('div', class_='wprm-recipe-instruction-group')

    for group in groups_instructions:
        groupname = group.find('h4', class_='wprm-recipe-group-name')
        instructions_group = group.find_all('ul', class_='wprm-recipe-instructions')
        
        if groupname :
            instructions.append(f"GroupName : {groupname.text}")
            for instruction in instructions_group:
                instruction_items = instruction.find_all('li', class_='wprm-recipe-instruction')
                # instructions = [f"{i+1}. {item.text.strip()}" for i, item in enumerate(instruction_items)]
                instructions.extend([f"{i+1}. {item.text.strip()}" for i, item in enumerate(instruction_items)])
        else :
            if instructions_container:
                instruction_items = instructions_container.find_all('li', class_='wprm-recipe-instruction')
                instructions = [f"{i+1}. {item.text.strip()}" for i, item in enumerate(instruction_items)]
    
    return "\n".join(instructions)

In [209]:
extract_instructions(soup)

'1. Preheat the oven to 350F.\n2. In a large mixing bowl, combine the sugar and eggs and whisk until well combined. Sprinkle in the flour, then add the lemon zest, salt, and nutmeg, and whisk to combine.\n3. Pour in the buttermilk, lemon juice, and vanilla extract, then mix well.\n4. Finally, whisk in the melted butter. Pour the filling into the prepared pie crust. (You can brush the edge with an egg wash for a more golden crust if desired.)\n5. Bake for about 40 minutes or until the crust is golden brown and the center barely wobbles when gently shaken. Loosely wrap the edge with foil after about 20 minutes of baking if they appear to be browning too quickly.\n6. Allow to cool completely before slicing. Top each serving with a dollop of whipped cream.'

In [182]:
ingredients_container = soup.find('div', class_='wprm-recipe-ingredients-container')
titles = ingredients_container.find_all('h4', class_='wprm-recipe-group-name')
ingredients_container.find_all('li', class_='wprm-recipe-ingredient')
ingredient_items = ingredients_container.find_all('li', class_='wprm-recipe-ingredient')

# extract_ingredients(soup)
for title in titles :
    print(title.text)
print(len(ingredient_items))
groups = ingredients_container.find_all('div', class_='wprm-recipe-ingredient-group')
print(len(groups))

for group in groups:
    groupname = group.find('h4', class_='wprm-recipe-group-name')
    ingredients = group.find_all('li', class_='wprm-recipe-ingredient')
    print(groupname)
    # for ingredient in ingredients:
    #     print(f"ingredient.text : {ingredient.text} : {groupname.text}") 


INGREDIENTS
For the Pie Crust
For the Filling
For the Whipped Cream Topping
18
4
<h4 class="wprm-recipe-group-name wprm-recipe-ingredient-group-name wprm-block-text-bold">INGREDIENTS</h4>
<h4 class="wprm-recipe-group-name wprm-recipe-ingredient-group-name wprm-block-text-bold">For the Pie Crust</h4>
<h4 class="wprm-recipe-group-name wprm-recipe-ingredient-group-name wprm-block-text-bold">For the Filling</h4>
<h4 class="wprm-recipe-group-name wprm-recipe-ingredient-group-name wprm-block-text-bold">For the Whipped Cream Topping</h4>


INSTRUCTIONS
For the Pie Crust
For the Filling
 For the Whipped Cream Topping:
For the Assembly


''

In [76]:
import re

url = "https://preppykitchen.com/wp-content/uploads/2024/12/Banana-Waffles-Recipe-Card-330x330.jpg"
match = re.search(r'([^/]+\.jpg)$', url)

if match:
    jpg_name = match.group(1)
    print(jpg_name)  # Output: Banana-Waffles-Recipe-Card-330x330.jpg
else:
    print("No match found")


Banana-Waffles-Recipe-Card-330x330.jpg
